In [ ]:
##########################################################################################
################################################################ Указать название скрипта
script = "__template-associate-ym.ipynb"

In [ ]:
##################################################################################
################################################################ импорт библиотек
import sys
import os
import configparser
import pandas as pd

# windows 10
fpath = os.path.join(os.path.dirname(script), 'patterns') 
sys.path.append(fpath)
config = configparser.ConfigParser()
config.read("__fields.ini")
config_db = configparser.ConfigParser()
config_db.read("__settings.ini")

# linux server
# sys.path.append(r'/var/home/rb1/!py_script/patterns')
# sys.path.append(r'/var/home/rb1/!py_script')
# config = configparser.ConfigParser()
# config.read(r'/var/home/rb1/!py_script/data_fields.ini')
# config_db = configparser.ConfigParser()
# config_db.read(r'/var/home/rb1/!py_script/settings.ini')

import ym
import yesterday
import numpy as np

from sqlalchemy import create_engine
from google.oauth2 import service_account

In [ ]:
###########################################################################################
################################################################ укажите параметры скрипта
dateFrom = "2022-01-04" # начальная дата
dateTo = str(yesterday.getYesterday()) #конечная дата
dataset = "optimize_mst_riverstart" # целевой датасет или база данных для записи

mode_write = "replace" # перезапись данных
#mode_write = "append" # добавление данных

# GoogleBigQuery - коннект
# project_id = 'powerbi4rb'
# json_path = r"patterns\\pbi4rb_gbq.json" # windows 10
# json_path = r'/var/home/rb1/!py_script/patterns/pbi4rb_gbq.json' # linux

# PostgreSQL - коннект
engine = create_engine("postgresql://" + config_db['postgresql']['username'] + ':' + config_db['postgresql']['password'] + '@' + config_db['postgresql']['host'] + ':' + config_db['postgresql']['port'] + "/" + dataset)

In [ ]:
########################################################################################
################################################################ Итоговые датафреймы (в них собираются данные)
res_df_assists_raw = pd.DataFrame()

In [ ]:
###############################################################################
############################################################### Яндекс метрика
token_ym = "XXXXXXXXXXXXXXXX"
clientLogin = 'chery'
counter = '88269306'
goals_metrika = ['290534052','290532335','290531916']

In [ ]:
ass_camp = ym.associate_conv_camp(counter,token_ym,dateFrom,dateTo,goals_metrika)
ass_sour = ym.associate_conv_sour(counter,token_ym,dateFrom,dateTo,goals_metrika)

In [ ]:
# ###########################################################################################
# ############################################### Сбор данных Метрики - Ассоциированные конверсии - часть 1

fields = 'assoc_stats_camp'
report_fields = config['yandex_metrika_logs'][fields]

metrika_dataset = ym.get_logs_stat(counter,token_ym,report_fields,dateFrom,dateTo)

# транспонируем столбец с конверсиями в несколько новых столбцов
for i in goals_metrika:
    metrika_dataset['Conversion_' + i] = np.where(metrika_dataset['ym_s_goalsID'].str.contains(i), 1, 0)
    print(i, ' - done')

res_df_assists_raw = pd.concat([res_df_assists_raw, metrika_dataset], ignore_index=True)

In [ ]:
columns_list = res_df_assists_raw.columns.tolist()
filtered_columns = list(filter(lambda x: 'Conversion_' in x, columns_list))
print(filtered_columns)

In [ ]:
goal_id = ['Conversion_138806242','Conversion_49892242','Conversion_223582376','Conversion_249940396']

In [ ]:
ass_camp = pd.DataFrame()

for i in goal_id:
    count_session = res_df_assists_raw.groupby('ym_s_clientID').agg(
    session = ('ym_s_clientID', 'count'),
    goal = (i, 'sum')
    ).reset_index()

    conversion_date = res_df_assists_raw[res_df_assists_raw[i] == 1].groupby('ym_s_clientID').agg(
    client_id=('ym_s_clientID', 'first'),
    conversion_date=('ym_s_date', 'max')
    ).reset_index(drop=True)

    merged = pd.merge(res_df_assists_raw, count_session, on='ym_s_clientID', how='left')
    merged = merged.rename(columns={'ym_s_clientID':'client_id'})
    merged = pd.merge(merged, conversion_date, on='client_id', how='left')
    merged['associated_conversion'] = merged[i].apply(lambda x: 0 if x == 1 else 1)
    merged['atr_window'] = (pd.to_datetime(merged['conversion_date']) - pd.to_datetime(merged['ym_s_date'])).dt.days

    filtered = merged.loc[(merged['goal'] > 0) & (merged[i] == 0) & 
                          (merged['ym_s_UTMMedium'] != '') & (merged['session'] > 1) & 
                          (merged['atr_window'] >= 0)]
    
    filtered['utm_campaign'] = filtered['ym_s_lastDirectClickOrder'].where(filtered['ym_s_lastDirectClickOrder'] != '', filtered['ym_s_UTMCampaign'])
 
    filtered = filtered.rename(columns={'ym_s_date':'date',
                                        'ym_s_UTMSource':'utm_source',
                                        'ym_s_UTMMedium':'utm_medium'})

    final = filtered
    result = final.groupby(['date', 'utm_source', 'utm_medium', 'utm_campaign']).agg({'associated_conversion': 'sum'}).reset_index()
    result = result.sort_values('date', ascending=False)

    if len(result) > 0:
        result[f'АК по цели: {i}'] = result['associated_conversion']
        ass_camp = pd.concat([ass_camp, result], ignore_index=True)

    sm = ass_camp.groupby(['date', 'utm_medium', 'utm_source', 'utm_campaign']).sum().reset_index()
    ass_camp = ass_camp.groupby(['date', 'utm_medium', 'utm_source', 'utm_campaign']).sum().reset_index()
    ass_camp.drop('associated_conversion', axis=1, inplace=True)

In [ ]:
res_df_assists_source = pd.DataFrame()

In [ ]:
# ###########################################################################################
# ############################################### Сбор данных Метрики - Ассоциированные конверсии - часть 1

fields = 'assoc_stats_source'
report_fields = config['yandex_metrika_logs'][fields]

metrika_dataset = ym.get_logs_stat(counter,token_ym,report_fields,dateFrom,dateTo)

# транспонируем столбец с конверсиями в несколько новых столбцов
for i in goals_metrika:
    metrika_dataset['Conversion_' + i] = np.where(metrika_dataset['ym_s_goalsID'].str.contains(i), 1, 0)
    print(i, ' - done')

res_df_assists_source = pd.concat([res_df_assists_source, metrika_dataset], ignore_index=True)

In [ ]:
goal_id = ['Conversion_290534052','Conversion_290532335','Conversion_290531916']

In [ ]:
ass_source = pd.DataFrame()
ass_adv_source = pd.DataFrame()
ad = pd.DataFrame()
direct = pd.DataFrame()
internal = pd.DataFrame()
messenger = pd.DataFrame()
organic = pd.DataFrame()
recommend = pd.DataFrame()
referral = pd.DataFrame()
saved = pd.DataFrame()
social = pd.DataFrame()

# ad
ad = res_df_assists_source.loc[(res_df_assists_source['ym_s_lastTrafficSource'] == 'ad')]
ad = ad[['ym_s_date','ym_s_lastTrafficSource','ym_s_lastAdvEngine','ym_s_visitID','ym_s_clientID','ym_s_goalsID']]
ad = ad.rename(columns={'ym_s_lastAdvEngine':'ym_s_source2'})

# direct
direct = res_df_assists_source.loc[(res_df_assists_source['ym_s_lastTrafficSource'] == 'direct')]
direct = direct[['ym_s_date','ym_s_lastTrafficSource','ym_s_visitID','ym_s_clientID','ym_s_goalsID']]

# internal
internal = res_df_assists_source.loc[(res_df_assists_source['ym_s_lastTrafficSource'] == 'internal')]
internal = internal[['ym_s_date','ym_s_lastTrafficSource','ym_s_visitID','ym_s_clientID','ym_s_goalsID']]

# messenger
messenger = res_df_assists_source.loc[(res_df_assists_source['ym_s_lastTrafficSource'] == 'messenger')]
messenger = messenger[['ym_s_date','ym_s_lastTrafficSource','ym_s_referer','ym_s_visitID','ym_s_clientID','ym_s_goalsID']]
messenger = messenger.rename(columns={'ym_s_referer':'ym_s_source2'})

# organic
organic = res_df_assists_source.loc[(res_df_assists_source['ym_s_lastTrafficSource'] == 'organic')]
organic = organic[['ym_s_date','ym_s_lastTrafficSource','ym_s_lastSearchEngineRoot','ym_s_visitID','ym_s_clientID','ym_s_goalsID']]
organic = organic.rename(columns={'ym_s_lastSearchEngineRoot':'ym_s_source2'})

# recommend
recommend = res_df_assists_source.loc[(res_df_assists_source['ym_s_lastTrafficSource'] == 'recommend')]
recommend = recommend[['ym_s_date','ym_s_lastTrafficSource','ym_s_lastReferalSource','ym_s_visitID','ym_s_clientID','ym_s_goalsID']]
recommend = recommend.rename(columns={'ym_s_lastReferalSource':'ym_s_source2'})

# referral
referral = res_df_assists_source.loc[(res_df_assists_source['ym_s_lastTrafficSource'] == 'referral')]
referral = referral[['ym_s_date','ym_s_lastTrafficSource','ym_s_lastReferalSource','ym_s_visitID','ym_s_clientID','ym_s_goalsID']]
referral = referral.rename(columns={'ym_s_lastReferalSource':'ym_s_source2'})

# social
social = res_df_assists_source.loc[(res_df_assists_source['ym_s_lastTrafficSource'] == 'social')]
social = social[['ym_s_date','ym_s_lastTrafficSource','ym_s_lastSocialNetwork','ym_s_visitID','ym_s_clientID','ym_s_goalsID']]
social = social.rename(columns={'ym_s_lastSocialNetwork':'ym_s_source2'})

ass_adv_source = pd.DataFrame()
ass_adv_source = pd.concat([ass_adv_source, ad], ignore_index=True)
ass_adv_source = pd.concat([ass_adv_source, direct], ignore_index=True)
ass_adv_source = pd.concat([ass_adv_source, internal], ignore_index=True)
ass_adv_source = pd.concat([ass_adv_source, messenger], ignore_index=True)
ass_adv_source = pd.concat([ass_adv_source, organic], ignore_index=True)
ass_adv_source = pd.concat([ass_adv_source, recommend], ignore_index=True)
ass_adv_source = pd.concat([ass_adv_source, referral], ignore_index=True)
ass_adv_source = pd.concat([ass_adv_source, social], ignore_index=True)

for i in goals_metrika:
    ass_adv_source['Conversion_' + i] = np.where(ass_adv_source['ym_s_goalsID'].str.contains(i), 1, 0)
    print(i, ' - done')


for i in goal_id:
    count_session = ass_adv_source.groupby('ym_s_clientID').agg(
    session = ('ym_s_clientID', 'count'),
    goal = (i, 'sum')
    ).reset_index()

    conversion_date = ass_adv_source[ass_adv_source[i] == 1].groupby('ym_s_clientID').agg(
    client_id=('ym_s_clientID', 'first'),
    conversion_date=('ym_s_date', 'max')
    ).reset_index(drop=True)

    merged = pd.merge(ass_adv_source, count_session, on='ym_s_clientID', how='left')
    merged = merged.rename(columns={'ym_s_clientID':'client_id'})
    merged = pd.merge(merged, conversion_date, on='client_id', how='left')
    merged['associated_conversion'] = merged[i].apply(lambda x: 0 if x == 1 else 1)
    merged['atr_window'] = (pd.to_datetime(merged['conversion_date']) - pd.to_datetime(merged['ym_s_date'])).dt.days

    filtered = merged.loc[(merged['goal'] > 0) & (merged[i] == 0) & 
                          (merged['session'] > 1) & 
                          (merged['atr_window'] >= 0)]
    
    filtered = filtered.rename(columns={'ym_s_date':'date',
                                        'ym_s_lastTrafficSource':'lastTrafficSource',
                                        'ym_s_source2':'source2'})

    final = filtered
    result = final.groupby(['date', 'lastTrafficSource', 'source2']).agg({'associated_conversion': 'sum'}).reset_index()
    result = result.sort_values('date', ascending=False)

    if len(result) > 0:
        result[f'АК по цели: {i}'] = result['associated_conversion']
        ass_source = pd.concat([ass_source, result], ignore_index=True)

    ass_source = ass_source.groupby(['date', 'lastTrafficSource', 'source2']).sum().reset_index()
    ass_source.drop('associated_conversion', axis=1, inplace=True)

In [ ]:
###################################################################################################
################################################################ Отправляем полученные данные - POSTGRESQL
config = configparser.ConfigParser()
config.read("settings.ini")

# PostgreSQL - коннект
engine = create_engine("postgresql://" + config['postgresql']['username'] + ':' + config['postgresql']['password'] + '@' + config['postgresql']['host'] + ':' + config['postgresql']['port'] + "/" + dataset)


res_df_df_ads_text.to_sql("yandex_ads", engine, if_exists=mode_write)
